<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto </font></h1>

## Introduction
In this project, we will explore, segment, and cluster the neighborhoods in the city of Toronto. A Wikipedia page exists that has all the information for Toronto neighborhood data.First, we will scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, we can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

## Install required libraries

In [2]:
!pip install --upgrade beautifulsoup4
!pip install --upgrade geopy
!pip install --upgrade folium
!pip install --upgrade geocoder

     |████████████████████████████████| 122kB 19.2MB/s eta 0:00:01
  Found existing installation: beautifulsoup4 4.7.1
    Uninstalling beautifulsoup4-4.7.1:
      Successfully uninstalled beautifulsoup4-4.7.1
     |████████████████████████████████| 122kB 11.7MB/s eta 0:00:01
  Found existing installation: geopy 1.18.1
    Uninstalling geopy-1.18.1:
      Successfully uninstalled geopy-1.18.1
     |████████████████████████████████| 102kB 21.8MB/s ta 0:00:01
     |████████████████████████████████| 102kB 16.7MB/s ta 0:00:01


## Import required libraries

In [3]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim     # convert an address into latitude and longitude values
import requests                           # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans 
from bs4 import BeautifulSoup
import xml 
import folium                             # map rendering library

print('Libraries imported.')

Libraries imported.


## Load Toronto Neighborhood Data in a dataframe: 
Scrap data from Wikipedia using BeautifulSoup

In [4]:
url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url,'lxml')

In [5]:
Toronto_table = soup.find('table')
postcode = []
borough  = []
neighbourhood = []

#------------metdhod (1)------------------------------------------------
# cell = Toronto_table.find_all('td')
# for i in range(0, len(cell), 3):
#     postcode.append(cell[i].text.strip())    # can replace .text.strip() with .find(text=True) that gives the same results
#     borough .append(cell[i+1].text.strip())
#     neighbourhood.append(cell[i+2].text.strip())
# df = pd.DataFrame([postcode, borough, neighbourhood]).transpose()
# df.columns = ['Postcode', 'Borough', 'Neighbourhood']
# df.head(10)


#------------metdhod (2)------------------------------------------------
# for row in Toronto_table.find_all('tr'):
#     cell = row.find_all('td')
#     if len(data)==3:
#         postcode.append(cell[0].text.strip())    # can replace .text.strip() with .find(text=True) that gives the same results
#         borough .append(cell[1].text.strip())
#         neighbourhood.append(cell[2].text.strip())    
        
# df = pd.DataFrame([postcode, borough, neighbourhood]).transpose()
# df.columns = ['Postcode', 'Borough', 'Neighbourhood']
# df.head(10)

#------------metdhod (3)------------------------------------------------
#dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)
# Search all the postcode, borough, neighborhood 
for tr in Toronto_table.find_all('tr'):
    row_data=[]
    for td in tr.find_all('td'):
        row_data.append(td.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 0 to 179
Data columns (total 3 columns):
Postalcode      180 non-null object
Borough         180 non-null object
Neighborhood    180 non-null object
dtypes: object(3)
memory usage: 5.6+ KB


## Data Cleaning

In [6]:
df['Borough'].replace('Not assigned', np.nan, inplace=True)
df.dropna(subset=['Borough'], inplace=True)
df['Neighborhood'].replace('', df['Borough'], inplace=True)
df.head()
df.info()

# df=df[df['Borough']!='Not assigned']
# df[df['Neighborhood']==" "]=df['Borough']
# df.head()
# df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 103 entries, 2 to 178
Data columns (total 3 columns):
Postalcode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 8.2+ KB


In [7]:
df_grouped = df.groupby(['Postalcode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
df_grouped.columns = ['Postalcode', 'Borough', 'Neighborhood']
df_grouped.info()

# df_grouped=df.groupby(['Postalcode', 'Borough'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x)).reset_index(drop=False)
# df_grouped.rename(columns={'Neighborhood':'Neighborhood_joined'},inplace=True)
# df_grouped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 3 columns):
Postalcode      103 non-null object
Borough         103 non-null object
Neighborhood    103 non-null object
dtypes: object(3)
memory usage: 2.5+ KB


In [8]:
df_grouped['Neighborhood'].replace('Not assigned', "Queen's Park", inplace=True)

df_grouped

,Postalcode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [9]:
df_grouped.shape

(103, 3)

## Geospatial Data

In [10]:
# import geocoder
# def get_geocoder(postal_code_from_df):
#      # initialize your variable to None
#      lat_lng_coords = None
#      # loop until you get the coordinates
#      while(lat_lng_coords is None):
#        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code_from_df))
#        lat_lng_coords = g.latlng
#      latitude = lat_lng_coords[0]
#      longitude = lat_lng_coords[1]
#      return latitude,longitude
# df_position = df_grouped
# for i in range(0,len(df_position)):
#     #df_position['Latitude'][i],df_position['Longitude'][i]=get_geocoder(df_grouped.iloc[i]['Postalcode']
#    df_position['Latitude'][i],df_position['Longitude'][i]=get_geocoder(df_position['Postalcode'].values)   # using np.vectorize to speed up.                                                                 
#    #df_position['Latitude'], df_position['Longitude'] = zip(*df_position['Postalcode'].apply(get_geocoder))
   


In [11]:
# csv file that has the geographical coordinates of each postal code
df_geospatial = pd.read_csv('http://cocl.us/Geospatial_data')
df_geospatial.columns = ['Postalcode', 'Latitude', 'Longitude']

In [12]:
df_position = pd.merge(df_grouped, df_geospatial, on=['Postalcode'], how='inner')
df_toronto = df_position[['Borough', 'Neighborhood', 'Postalcode', 'Latitude', 'Longitude']].copy()
df_toronto.head()

,Borough,Neighborhood,Postalcode,Latitude,Longitude
0,Scarborough,"Malvern, Rouge",M1B,43.806686,-79.194353
1,Scarborough,"Rouge Hill, Port Union, Highland Creek",M1C,43.784535,-79.160497
2,Scarborough,"Guildwood, Morningside, West Hill",M1E,43.763573,-79.188711
3,Scarborough,Woburn,M1G,43.770992,-79.216917
4,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [13]:
address = 'Toronto, Canada'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the City of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


The geograpical coordinate of the City of Toronto are 43.6534817, -79.3839347.


In [14]:
# creating map of Toronto using latitude, longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# adding markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3199cc',
        fill_opacity=0.3,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [15]:
df_toronto.loc[0, 'Neighborhood'] #exploring first neighbourhood

'Malvern, Rouge'

In [16]:
neighborhood_latitude = df_toronto.loc[0, 'Latitude']   # neighborhood latitude value
neighborhood_longitude = df_toronto.loc[0, 'Longitude'] # neighborhood longitude value
neighborhood_name = df_toronto.loc[0, 'Neighborhood']   # neighborhood name
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Malvern, Rouge are 43.806686299999996, -79.19435340000001.


#### Define Foursquare Credentials and Version

In [17]:
# @hidden_cell
CLIENT_ID = '2KSOWMNUTD3UQMGXDA3F0QRZHEWBCCA03X4IOJY0PPT21CYV' # your Foursquare ID
CLIENT_SECRET = 'STUNTIEU53FRY0NBEARBDSZAQVH0KCXX1SQJQWMZMKMI1UHZ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2KSOWMNUTD3UQMGXDA3F0QRZHEWBCCA03X4IOJY0PPT21CYV
CLIENT_SECRET:STUNTIEU53FRY0NBEARBDSZAQVH0KCXX1SQJQWMZMKMI1UHZ


In [18]:
LIMIT = 100
radius = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=2KSOWMNUTD3UQMGXDA3F0QRZHEWBCCA03X4IOJY0PPT21CYV&client_secret=STUNTIEU53FRY0NBEARBDSZAQVH0KCXX1SQJQWMZMKMI1UHZ&v=20180605&ll=43.806686299999996,-79.19435340000001&radius=500&limit=100'

In [19]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ecd6c9871c428001bce5db7'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Malvern',
  'headerFullLocation': 'Malvern, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 2,
  'suggestedBounds': {'ne': {'lat': 43.8111863045, 'lng': -79.18812958073042},
   'sw': {'lat': 43.80218629549999, 'lng': -79.2005772192696}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb6b9446edc76b0d771311c',
       'name': 'Wendy’s',
       'location': {'crossStreet': 'Morningside & Sheppard',
        'lat': 43.80744841934756,
        'lng': -79.19905558052072,
        'labeledLatLngs': [{'label': 'display'

In [20]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [21]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Wendy’s,Fast Food Restaurant,43.807448,-79.199056
1,Interprovincial Group,Print Shop,43.805630,-79.200378


In [22]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

2 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto

#### Let's create a function to repeat the same process to all the neighborhoods in Toronto

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [29]:
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'],
                                   latitudes=df_toronto['Latitude'],
                                   longitudes=df_toronto['Longitude'],
                                   radius=500)

Malvern, Rouge
Rouge Hill, Port Union, Highland Creek
Guildwood, Morningside, West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park, Ionview, East Birchmount Park
Golden Mile, Clairlea, Oakridge
Cliffside, Cliffcrest, Scarborough Village West
Birch Cliff, Cliffside West
Dorset Park, Wexford Heights, Scarborough Town Centre
Wexford, Maryvale
Agincourt
Clarks Corners, Tam O'Shanter, Sullivan
Milliken, Agincourt North, Steeles East, L'Amoreaux East
Steeles West, L'Amoreaux West
Upper Rouge
Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
York Mills, Silver Hills
Willowdale, Newtonbrook
Willowdale, Willowdale East
York Mills West
Willowdale, Willowdale West
Parkwoods
Don Mills
Don Mills
Bathurst Manor, Wilson Heights, Downsview North
Northwood Park, York University
Downsview
Downsview
Downsview
Downsview
Victoria Village
Parkview Hill, Woodbine Gardens
Woodbine Heights
The Beaches
Leaside
Thorncliffe Park
East Toronto, Broadview North (Old East York)
The Danforth West, 